# Predict NER Model

In [8]:
requirements = [["spacy", "3.0.6"]]

In [9]:
import spacy

In [11]:
# my_bucket = "DH-SECURE-OCHATTER"
# conn = boto3.client(service_name='s3',
#         aws_access_key_id= os.environ.get("S3_ACCESS_KEY"),
#         aws_secret_access_key= os.environ.get("S3_SECRET_KEY"),
#         endpoint_url=os.environ.get("S3_ENDPOINT_URL"))

In [28]:
model_name = "en_core_web_sm-3.0.0"
model_path = "model/"

In [29]:
# file = tarfile.open(model_path + model_name + ".tar.gz")
# file.extractall(model_path)
# file.close()

In [30]:
nlp = spacy.load(model_path + model_name)

In [31]:
print("Loaded model '%s'" % model_name)

Loaded model 'en_core_web_sm-3.0.0'


## Predictor and Validator

In [13]:
def predictor(feed_text):
    
    doc = nlp(feed_text)
    return [(ent.text, ent.label_) for ent in doc.ents]

In [22]:
def validator(x):
    
    return type(x) == str